In [1]:
import polars as pl
import pandas as pd

### From:
  * https://towardsdatascience.com/understanding-groupby-in-polars-dataframe-by-examples-1e910e4095b3

Zone,School,Name,Science,Math
str,str,str,i64,i64
"""North""","""Rushmore""","""Jonny""",78,70
"""North""","""Rushmore""","""Mary""",39,45
"""South""","""Bayside""","""Joe""",76,68
"""South""","""Rydell""","""Jakob""",56,90
"""East""","""Shermer""","""Jimmy""",67,45
"""East""","""Shermer""","""Erik""",89,66
"""West""","""Ridgemont""","""Lam""",100,89
"""West""","""Hogwarts""","""Yip""",55,32


In [9]:
q = (
    pl.scan_csv("csv_data.csv")
    .filter(pl.col("year") > 1999)
    .groupby("make")
    .agg(pl.all().len())
)

df = q.collect()

In [10]:
df.head()

make,year,model
str,u32,u32
"""PEUGEOT""",74,74
"""FREIGHTLINER""",226,226
"""E-TON""",107,107
"""SAAB""",32,32
"""CHRYSLER""",102,102


In [24]:
scores = {'Zone': ['North', 'North', 'South', 'South',
                   'East', 'East', 'West', 'West'], 
          'School': ['Rushmore', 'Rushmore','Bayside','Rydell',
                     'Shermer','Shermer','Ridgemont','Hogwarts'],
          'Name': ['Jonny', 'Mary', 'Joe', 'Jakob', 
                   'Jimmy', 'Erik', 'Lam', 'Yip'], 
          'Math': [78, 39, 76, 56, 67, 89, 100, 55],
          'Science': [70, 45, 68, 90, 45, 66, 89, 32]}
df = pl.DataFrame(scores, columns = 
                  ['Zone', 'School', 'Name', 
                   'Science', 'Math'])          
df

Zone,School,Name,Science,Math
str,str,str,i64,i64
"""North""","""Rushmore""","""Jonny""",78,70
"""North""","""Rushmore""","""Mary""",39,45
"""South""","""Bayside""","""Joe""",76,68
"""South""","""Rydell""","""Jakob""",56,90
"""East""","""Shermer""","""Jimmy""",67,45
"""East""","""Shermer""","""Erik""",89,66
"""West""","""Ridgemont""","""Lam""",100,89
"""West""","""Hogwarts""","""Yip""",55,32


In [27]:
q = (
    df.lazy().groupby(pl.col("Zone")).agg(pl.col("School"))
)
pp_df = q.collect()

In [28]:
pp_df.head()

Zone,School
str,list[str]
"""West""","[""Ridgemont"", ""Hogwarts""]"
"""East""","[""Shermer"", ""Shermer""]"
"""North""","[""Rushmore"", ""Rushmore""]"
"""South""","[""Bayside"", ""Rydell""]"


In [31]:
q = (
    df.lazy().groupby(pl.col("Zone")).
    agg(pl.col("School")).
    filter(
        pl.col('Zone')=='East'        
    )
)
q.collect()

Zone,School
str,list[str]
"""East""","[""Shermer"", ""Shermer""]"


In [33]:
q = (
    df
    .lazy()
    .groupby(pl.col("Zone"))
    .agg(        
        #pl.col('Science').max().alias('Science(Max)')
        pl.max('Science').alias('Science(Max)')
    )    
)
q.collect()

Zone,Science(Max)
str,i64
"""North""",78
"""East""",89
"""South""",76
"""West""",100


In [34]:
q = (
    df
    .lazy()
    .groupby(pl.col("Zone"))
    .agg(        
        [
            pl.col('Science').count().alias('Number of Schools'),
            pl.col('Science').max().alias('Science(Max)')
        ]
    )    
)
q.collect()

Zone,Number of Schools,Science(Max)
str,u32,i64
"""North""",2,78
"""West""",2,100
"""South""",2,76
"""East""",2,89


In [35]:
q = (
    df
    .lazy()
    .groupby(pl.col("Zone"))
    .agg(        
        [
            pl.col('Science').count().alias('Number of Schools'),
            pl.col('Science').max().alias('Science(Max)'),
            pl.col('Science').min().alias('Science(Min)'),
            pl.col('Science').mean().alias('Science(Mean)'),
            pl.col('Math').max().alias('Math(Max)'),
            pl.col('Math').min().alias('Math(Min)'),    
            pl.col('Math').mean().alias('Math(Mean)'),    
        ]
    )    
)
q.collect()

Zone,Number of Schools,Science(Max),Science(Min),Science(Mean),Math(Max),Math(Min),Math(Mean)
str,u32,i64,i64,f64,i64,i64,f64
"""East""",2,89,67,78.0,66,45,55.5
"""North""",2,78,39,58.5,70,45,57.5
"""West""",2,100,55,77.5,89,32,60.5
"""South""",2,76,56,66.0,90,68,79.0


In [36]:
q = (
    df
    .lazy()
    .groupby(pl.col("Zone"))
    .agg(        
        [
            pl.max('Science').alias('Science(Max)')
        ]
    )    
    .sort(by='Zone')
)
q.collect()

Zone,Science(Max)
str,i64
"""East""",89
"""North""",78
"""South""",76
"""West""",100


Observe that the Zone is sorted alphabetically, and not based on the cardinal directions (i.e. North, South, East, and West). So how do you sort the Zone based on cardinal directions? The following code snippet shows how this can be done:

In [37]:
df_sortorder = pl.DataFrame({
   'Zone' : ['North','South','East','West'],
   'Zone_order' : [0,1,2,3]
}).lazy()
q = (
    df
    .lazy()
    .join(df_sortorder, on='Zone', how='left')
    .groupby(by=['Zone','Zone_order'])
    .agg(
        [
            pl.max('Science').alias('Science(Max)')
        ]
    )
    .sort('Zone_order')
    .select(
        pl.exclude('Zone_order')
    )
)
q.collect()

Zone,Science(Max)
str,i64
"""North""",78
"""South""",76
"""East""",89
"""West""",100


In [38]:
q = (
    pl.scan_csv("csv_data.csv")
    .filter(pl.col("year") > 1999)
    .lazy()
    .groupby("make")
    .agg(pl.all().len())
)

df = q.collect()

#df.lazy().groupby(by='Zone').agg('School')

In [39]:
q = (
    pl.scan_csv('csv_data.csv')
    .groupby(by=['year','make'])
    .agg(
        pl.col(['make']).count().alias('count')        
    )
    .sort(by=['year','make'])
)
q.collect()

year,make,count
i64,str,u32
2001,"""ACURA""",7
2001,"""AM GENERAL""",1
2001,"""AMERICAN IRONH...",7
2001,"""APRILIA""",16
2001,"""ARCTIC CAT""",51
2001,"""ASTON MARTIN""",2
2001,"""ATK""",7
2001,"""AUDI""",13
2001,"""AVANTI""",1


In [40]:
q = (
    pl.scan_csv('csv_data.csv')
    .groupby(by=['year','make'])
    .agg(        
        pl.count()
    )
    .sort(by=['year','make'])
)
q.collect()

year,make,count
i64,str,u32
2001,"""ACURA""",7
2001,"""AM GENERAL""",1
2001,"""AMERICAN IRONH...",7
2001,"""APRILIA""",16
2001,"""ARCTIC CAT""",51
2001,"""ASTON MARTIN""",2
2001,"""ATK""",7
2001,"""AUDI""",13
2001,"""AVANTI""",1


### Showing the first and last model for each car make
If you want to show the first and last model for each car make, use the first() and last() methods, respectively:

In [43]:
q = (
    pl.scan_csv('csv_data.csv')
    .groupby(by=[pl.col("year"), pl.col("make")])
    .agg(
        [
            pl.col("model"),
            pl.first('model').alias('first'),
            pl.last('model').alias('last'),
        ]    
    )
    .sort(by=['year','make'])
)
q.collect()

year,make,model,first,last
i64,str,list[str],str,str
2001,"""ACURA""","[""CL"", ""EL"", ... ""TL""]","""CL""","""TL"""
2001,"""AM GENERAL""","[""HUMMER""]","""HUMMER""","""HUMMER"""
2001,"""AMERICAN IRONH...","[""CLASSIC"", ""LEGEND"", ... ""THUNDER""]","""CLASSIC""","""THUNDER"""
2001,"""APRILIA""","[""ATLANTIC 500"", ""ETV 1000 CAPONORD"", ... ""SL1000 FALCO""]","""ATLANTIC 500""","""SL1000 FALCO"""
2001,"""ARCTIC CAT""","[""250 2X4"", ""250 4X4"", ... ""ZRT 800 LE""]","""250 2X4""","""ZRT 800 LE"""
2001,"""ASTON MARTIN""","[""DB7"", ""VANQUISH""]","""DB7""","""VANQUISH"""
2001,"""ATK""","[""125 CC"", ""250 ENDURO"", ... ""605 ENDURO""]","""125 CC""","""605 ENDURO"""
2001,"""AUDI""","[""A3"", ""A4"", ... ""TT QUATTRO""]","""A3""","""TT QUATTRO"""
2001,"""AVANTI""","[""II""]","""II""","""II"""


In [17]:
df = pl.DataFrame(
    {
        "a": [None, 2, 3, 4],
        "b": [0.5, None, 2.5, 13],
        "c": [True, True, False, None],
    }
)
q = ( df.lazy().groupby("b").agg(pl.all().len()))
q.collect()

b,a,c
f64,u32,u32
0.5,1,1
2.5,1,1
13.0,1,1
null,1,1


In [44]:
q = (
    pl.scan_csv('insurance.csv')
)
q.collect()

age,sex,bmi,children,smoker,region,charges
i64,str,f64,i64,str,str,f64
19,"""female""",27.9,0,"""yes""","""southwest""",16884.924
18,"""male""",33.77,1,"""no""","""southeast""",1725.5523
28,"""male""",33.0,3,"""no""","""southeast""",4449.462
33,"""male""",22.705,0,"""no""","""northwest""",21984.47061
32,"""male""",28.88,0,"""no""","""northwest""",3866.8552
31,"""female""",25.74,0,"""no""","""southeast""",3756.6216
46,"""female""",33.44,1,"""no""","""southeast""",8240.5896
37,"""female""",27.74,3,"""no""","""northwest""",7281.5056
37,"""male""",29.83,2,"""no""","""northeast""",6406.4107


In [45]:
q = (
    pl.scan_csv('insurance.csv')
    .groupby(by='region')
    .agg(
        [
            (pl.col('sex') == 'male').sum().alias('male'),
            (pl.col('sex') == 'female').sum().alias('female'),
        ]    
    )
    .sort(by='region')
)
q.collect()

region,male,female
str,u32,u32
"""northeast""",163,161
"""northwest""",161,164
"""southeast""",189,175
"""southwest""",163,162


In [46]:
q = (
    pl.scan_csv('insurance.csv')
    .groupby(by='region')
    .agg(
        [
            (pl.col('charges')
             .filter(pl.col('sex')== 'male'))
            .mean()
            .alias('male_mean_charges'),
            
            (pl.col('charges')
             .filter(pl.col('sex')== 'female'))
            .mean()
            .alias('female_mean_charges'),
        ]    
    )
    .sort(by='region')
)
q.collect()

region,male_mean_charges,female_mean_charges
str,f64,f64
"""northeast""",13854.005374,12953.203151
"""northwest""",12354.119575,12479.870397
"""southeast""",15879.617173,13499.669243
"""southwest""",13412.883576,11274.411264


In [47]:
q = (
    pl.scan_csv('insurance.csv')
    .filter(
        (pl.col('region')=='northeast') & (pl.col('sex') == 'male')    
    )
    .select(
        pl.col('charges')
    )
    .mean()    
)
q.collect()

charges
f64
13854.005374


### Computing the proportion of smokers in each region
To calculate the proportion of smokers in each region, you need to:

  * sum up all the smokers
  * count the total number of rows in the smoker column
  
Here is the code snippet to perform the steps described above:



In [48]:
q = (
    pl.scan_csv('insurance.csv')
    .groupby(by='region')
    .agg(
        [
            ((pl.col('smoker')=='yes').sum() / 
             (pl.col('smoker')).count() * 100).alias('Smoker %')
        ]    
    )
    .sort(by='region')
)
q.collect()

region,Smoker %
str,f64
"""northeast""",20.679012
"""northwest""",17.846154
"""southeast""",25.0
"""southwest""",17.846154


In [49]:
pandas_df = q.collect().to_pandas()
pandas_df.head()

,region,Smoker %
0,northeast,20.679012
1,northwest,17.846154
2,southeast,25.000000
3,southwest,17.846154


In [ ]:
q = (
    df
    .lazy()
    .groupby(by='Zone')
    .agg(
        'School'
    )
    .filter(
        pl.col('Zone')=='East'        
    )
)
q.collect()

## From: 
  * https://stackoverflow.com/questions/71790235/polars-switching-between-dtypes-within-a-dataframe

### String to Int/Float

In [50]:
df = pl.DataFrame({"bar": ["100", "250", "125", ""]})

df.with_columns([
    pl.format("This is {}!", pl.col("bar"))
])

bar,literal
str,str
"""100""","""This is 100!"""
"""250""","""This is 250!"""
"""125""","""This is 125!"""
"""""","""This is !"""


### Int/Float to String
The same process can be used to convert numbers to strings - in this case, the utf8 datatype.

Let me modify your dataset slightly:

In [52]:
df = pl.DataFrame({"bar": [100.5, 250.25, 1250000, None]})
df.with_column(pl.col("bar").cast(pl.Utf8, strict=False).alias("bar_string"))

bar,bar_string
f64,str
100.5,"""100.5"""
250.25,"""250.25"""
1.25e6,"""1250000.0"""
null,null


In [53]:
df.with_column(
    pl.col("bar").apply(lambda x: f"This is ${x:,.2f}!").alias("bar_fstring")
)

bar,bar_fstring
f64,str
100.5,"""This is $100.5..."
250.25,"""This is $250.2..."
1.25e6,"""This is $1,250..."
null,null


In [51]:
df = pl.DataFrame({"bar": ["100", "250", "125", ""]})
df.with_column(pl.col('bar').cast(pl.Int64, strict=False).alias('bar_int'))

bar,bar_int
str,i64
"""100""",100
"""250""",250
"""125""",125
"""""",null


In [55]:
from datetime import datetime, timedelta, date

In [59]:
df3 = pl.DataFrame(
    {
        "dt": ["2022-09-03", "2022-09-02"],
        "x": [2.0, 5.0],
        "y": [2.5, 2.0],
    }
)  

In [60]:
df3.head()

dt,x,y
str,f64,f64
"""2022-09-03""",2.0,2.5
"""2022-09-02""",5.0,2.0


In [61]:
df3.with_column(pl.col("dt").str.strptime(pl.Datetime, fmt="%Y-%m-%d").cast(pl.Datetime))

dt,x,y
datetime[μs],f64,f64
2022-09-03 00:00:00,2.0,2.5
2022-09-02 00:00:00,5.0,2.0


In [62]:
dataset = pl.DataFrame({"date": ["2020-01-02", "2020-01-03", "2020-01-04"], "index": [1, 2, 3]})

q = dataset.lazy().with_column(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"))

df = q.collect()

In [63]:
df.head()

date,index
date,i64
2020-01-02,1
2020-01-03,2
2020-01-04,3


# JOIN Dataframes
  * https://pola-rs.github.io/polars-book/user-guide/quickstart/quick-exploration-guide.html

In [65]:
import numpy as np

In [66]:
df = pl.DataFrame({"a": np.arange(0, 8), 
                   "b": np.random.rand(8), 
                   "c": [datetime(2022, 12, 1) + timedelta(days=idx) for idx in range(8)],
                   "d": [1, 2.0, np.NaN, np.NaN, 0, -5, -42, None]
                  })

df2 = pl.DataFrame({
                    "x": np.arange(0, 8), 
                    "y": ['A', 'A', 'A', 'B', 'B', 'C', 'X', 'X'],
})


Our two DataFrames both have an 'id'-like column: a and x. We can use those columns to join the DataFrames in this example.

In [67]:
df.join(df2, left_on="a", right_on="x")

a,b,c,d,y
i64,f64,datetime[μs],f64,str
0,0.598546,2022-12-01 00:00:00,1.0,"""A"""
1,0.015414,2022-12-02 00:00:00,2.0,"""A"""
2,0.276842,2022-12-03 00:00:00,NaN,"""A"""
3,0.205707,2022-12-04 00:00:00,NaN,"""B"""
4,0.370403,2022-12-05 00:00:00,0.0,"""B"""
5,0.306095,2022-12-06 00:00:00,-5.0,"""C"""
6,0.286269,2022-12-07 00:00:00,-42.0,"""X"""
7,0.561046,2022-12-08 00:00:00,null,"""X"""


## Concat

We can also concatenate two DataFrames. Vertical concatenation will make the DataFrame longer. Horizontal concatenation will make the DataFrame wider. Below you can see the result of an horizontal concatenation of our two DataFrames.

In [68]:
pl.concat([df,df2], how="horizontal")


a,b,c,d,x,y
i64,f64,datetime[μs],f64,i64,str
0,0.598546,2022-12-01 00:00:00,1.0,0,"""A"""
1,0.015414,2022-12-02 00:00:00,2.0,1,"""A"""
2,0.276842,2022-12-03 00:00:00,NaN,2,"""A"""
3,0.205707,2022-12-04 00:00:00,NaN,3,"""B"""
4,0.370403,2022-12-05 00:00:00,0.0,4,"""B"""
5,0.306095,2022-12-06 00:00:00,-5.0,5,"""C"""
6,0.286269,2022-12-07 00:00:00,-42.0,6,"""X"""
7,0.561046,2022-12-08 00:00:00,null,7,"""X"""


## Missing data
This page sets out how missing data is represented in Polars and how missing data can be filled.

null and NaN values

In [69]:
df = pl.DataFrame(
    {
        "value": [1, None],
    },
)
print(df)


shape: (2, 1)
┌───────┐
│ value │
│ ---   │
│ i64   │
╞═══════╡
│ 1     │
│ null  │
└───────┘


In [70]:
null_count_df = df.null_count()
print(null_count_df)


shape: (1, 1)
┌───────┐
│ value │
│ ---   │
│ u32   │
╞═══════╡
│ 1     │
└───────┘


In [71]:
is_null_series = df.select(
    pl.col("value").is_null(),
)
print(is_null_series)


shape: (2, 1)
┌───────┐
│ value │
│ ---   │
│ bool  │
╞═══════╡
│ false │
│ true  │
└───────┘


## Filling missing data
Missing data in a Series can be filled with the fill_null method. You have to specify how you want the fill_null method to fill the missing data. The main ways to do this are filling with:

  * a literal such as 0 or "0"
  * a strategy such as filling forwards
  * an expression such as replacing with values from another column
  * interpolation
  
We illustrate each way to fill nulls by defining a simple DataFrame with a missing value in col2:

In [73]:
df = pl.DataFrame(
    {
        "col1": [1, 2, 3],
        "col2": [1, None, 3],
    },
)
print(df)


shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 1    │
│ 2    ┆ null │
│ 3    ┆ 3    │
└──────┴──────┘


In [74]:
fill_literal_df = (
    df.with_column(
        pl.col("col2").fill_null(
            pl.lit(2),
        ),
    ),
)
print(fill_literal_df)


(shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 1    │
│ 2    ┆ 2    │
│ 3    ┆ 3    │
└──────┴──────┘,)


## Treatment of NaN (not missing != NULL)

NaN values are considered to be a type of floating point data and are not considered to be missing data in Polars. This means:

NaN values are not counted with the null_count method
NaN values are filled when you use fill_nan method but are not filled with the fill_null method
Polars has is_nan and fill_nan methods which work in a similar way to the is_null and fill_null methods. The underlying Arrow arrays do not have a pre-computed validity bitmask for NaN values so this has to be computed for the is_nan method.

One further difference between null and NaN values is that taking the mean of a column with null values excludes the null values from the calculation but with NaN values taking the mean results in a NaN. This behaviour can be avoided by replacing the NaN values with null values;

In [75]:
nan_df = pl.DataFrame(
    {
        "value": [1.0, np.NaN, float("nan"), 3.0],
    },
)
print(nan_df)

shape: (4, 1)
┌───────┐
│ value │
│ ---   │
│ f64   │
╞═══════╡
│ 1.0   │
│ NaN   │
│ NaN   │
│ 3.0   │
└───────┘


In [76]:
mean_nan_df = nan_df.with_column(
    pl.col("value").fill_nan(None).alias("value"),
).mean()
print(mean_nan_df)


shape: (1, 1)
┌───────┐
│ value │
│ ---   │
│ f64   │
╞═══════╡
│ 2.0   │
└───────┘


## Pandas examples v. Polars

In [81]:
import pandas as pd
infile ='insurance.csv'
df = pd.read_csv(infile)
groupedDf = df.loc[:,['sex','charges']].groupby('sex').sum('charges')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [84]:
df = pd.DataFrame({
    "type": ["m", "n", "o", "m", "m", "n", "n"],
    "c": [1, 1, 1, 2, 2, 2, 2],
})

df["size"] = df.groupby("c")["type"].transform(len)
df.head()

,type,c,size
0,m,1,3
1,n,1,3
2,o,1,3
3,m,2,4
4,m,2,4


In [85]:
df = pl.DataFrame({
    "type": ["m", "n", "o", "m", "m", "n", "n"],
    "c": [1, 1, 1, 2, 2, 2, 2],
})

df.select([
    pl.all(),
    pl.col("type").count().over("c").alias("size")
])


type,c,size
str,i64,u32
"""m""",1,3
"""n""",1,3
"""o""",1,3
"""m""",2,4
"""m""",2,4
"""n""",2,4
"""n""",2,4


In [89]:
df = pl.DataFrame({
    "type": ["m", "n", "o", "m", "m", "n", "n"],
    "c": [1, 1, 1, 2, 2, 2, 2],
})

df.filter(
    (pl.col("type") == 'n') & (pl.col("c") > 1)
)


type,c
str,i64
"""n""",2
"""n""",2


## Adding columns

In [91]:
df = pd.DataFrame({
    "type": ["m", "n", "o", "m", "m", "n", "n"],
    "c": [1, 1, 1, 2, 2, 2, 2],
})
df["tenXValue"] = df["c"] * 10
df["hundredXValue"] = df["c"] * 100
df.head()

,type,c,tenXValue,hundredXValue
0,m,1,10,100
1,n,1,10,100
2,o,1,10,100
3,m,2,20,200
4,m,2,20,200


In [94]:
df = pl.DataFrame({
    "type": ["m", "n", "o", "m", "m", "n", "n"],
    "c": [1, 1, 1, 2, 2, 2, 2],
})
df = df.with_columns([
    (pl.col("c") * 10).alias("tenXValue"),
    (pl.col("c") * 100).alias("hundredXValue"),
])

df.head()

type,c,tenXValue,hundredXValue
str,i64,i64,i64
"""m""",1,10,100
"""n""",1,10,100
"""o""",1,10,100
"""m""",2,20,200
"""m""",2,20,200


In [83]:
df = pl.scan_csv(infile)
groupedDf = df.groupby('sex').agg(pl.col('charges').sum()).collect()
df.collect().head()

age,sex,bmi,children,smoker,region,charges
i64,str,f64,i64,str,str,f64
19,"""female""",27.9,0,"""yes""","""southwest""",16884.924
18,"""male""",33.77,1,"""no""","""southeast""",1725.5523
28,"""male""",33.0,3,"""no""","""southeast""",4449.462
33,"""male""",22.705,0,"""no""","""northwest""",21984.47061
32,"""male""",28.88,0,"""no""","""northwest""",3866.8552


## Polars DataFrame filter data in a period of time (start and end time)
  * https://stackoverflow.com/questions/73959629/polars-dataframe-filter-data-in-a-period-of-time-start-and-end-time


1


Is there a way to filter data in a period of time (i.e., start time and end time) using polars?

In [103]:
import pandas as pd
import polars as pl

dr = pd.date_range(start='2020-01-01', end='2021-01-01', freq="30min") 
df = pd.DataFrame({"timestamp": dr})
pf = pl.from_pandas(df)

pf.filter((pl.col("timestamp").dt.hour()>=9) & (pl.col("timestamp").dt.minute()>=30))

timestamp
datetime[ns]
2020-01-01 09:30:00
2020-01-01 10:30:00
2020-01-01 11:30:00
2020-01-01 12:30:00
2020-01-01 13:30:00
2020-01-01 14:30:00
2020-01-01 15:30:00
2020-01-01 16:30:00
2020-01-01 17:30:00


In [95]:
from datetime import datetime, time
import polars as pl

start = datetime(2020,1,1)
stop = datetime(2021,1,1)
df = pl.DataFrame({'timestamp':pl.date_range(low=start, high=stop, interval="30m")}) 

In [100]:
len(df)

17569

In [101]:
(
df
.filter(
        pl.col("timestamp").cast(pl.Time).is_between(
            time(9,30),time(16),include_bounds=True
            )
        )
)


timestamp
datetime[μs]
2020-01-01 09:30:00
2020-01-01 10:00:00
2020-01-01 10:30:00
2020-01-01 11:00:00
2020-01-01 11:30:00
2020-01-01 12:00:00
2020-01-01 12:30:00
2020-01-01 13:00:00
2020-01-01 13:30:00


In [104]:
start_date = "2022-03-22 00:00:00"
end_date = "2022-03-27 00:00:00"
df = pl.DataFrame(
    {
        "dates": [
                  "2022-03-22 00:00:00",
                  "2022-03-23 00:00:00",
                  "2022-03-24 00:00:00",
                  "2022-03-25 00:00:00",
                  "2022-03-26 00:00:00",
                  "2022-03-27 00:00:00",
                  "2022-03-28 00:00:00",
                 ]
    }
)
df.with_column(pl.col("dates").is_between(start_date,end_date)).filter(pl.col("is_between") == True)


dates,is_between
str,bool
"""2022-03-23 00:...",true
"""2022-03-24 00:...",true
"""2022-03-25 00:...",true
"""2022-03-26 00:...",true


In [105]:
df.filter(pl.col("dates").is_between(start_date, end_date))

dates
str
"""2022-03-23 00:..."
"""2022-03-24 00:..."
"""2022-03-25 00:..."
"""2022-03-26 00:..."


## from:
  * https://stackoverflow.com/questions/73212628/retrieve-date-from-datetime-column-in-polars

In [107]:
df = pl.DataFrame({
    'time': [datetime.now()]
})

df.with_column(
    pl.col("time").cast(pl.Date)
)

time
date
2023-01-02


In [99]:
df.__len__

<bound method DataFrame.__len__ of shape: (17569, 1)
┌─────────────────────┐
│ timestamp           │
│ ---                 │
│ datetime[μs]        │
╞═════════════════════╡
│ 2020-01-01 00:00:00 │
│ 2020-01-01 00:30:00 │
│ 2020-01-01 01:00:00 │
│ 2020-01-01 01:30:00 │
│ ...                 │
│ 2020-12-31 22:30:00 │
│ 2020-12-31 23:00:00 │
│ 2020-12-31 23:30:00 │
│ 2021-01-01 00:00:00 │
└─────────────────────┘>